In [56]:
import sys; sys.path.append('..')
from freesydney import *
from collections import UserString

In [57]:

class Speech(UserString):
    def __init__(self, data:str, who='', what='', how=''):
        super().__init__(data)
        self.who=who
        self.what=what
        self.how=how
    
    def __str__(self):
        return self.data

    def __repr__(self): 
        return str(self)

    def _repr_html_(self):
        how=f' (<i>{self.how}</i>)' if self.how else ''
        o = f'<p class="speech" style="margin-bottom:1em;"><b>{self.who.name.upper()}{how}</b>: {self.what}</p>'
        return o

    def save(self):
        if hasattr(self,'_convo') and self._convo is not None:
            self._convo.extend_dialogue([self])
    
    def parse(string):
        res = Speech.from_playscript(string)
        if res is not None: return res

    def from_playscript(line):
        if not line or not ':' in line: return
        whohow,what = line.split(':', 1)
        if '(' in whohow:
            who,how=whohow.split('(',1)
            who = who.strip()
            how = how.split(')',1)[0].strip()
        else:
            who = whohow.strip()
            how = ''

        if who and what:# and what.strip()[-1] in sentence_enders:
            return Speech(
                line,
                who=Agent(who.strip()),
                what=what.strip(),
                how=how.strip()
            )




In [46]:



class Speeches(UserList):
    def __init__(
        self, 
        data=[],
        string = '',
        string_prefix = '',
        string_suffix = '',
        sep='\n\n',
        _convo=None
        ):

        # init as list
        super().__init__(data)
        # set attrs
        self.string = string
        self.string_prefix = string_prefix
        self.string_suffix = string_suffix
        self.sep = sep

        self._agents = {}
        self._convo = _convo

    def _repr_html_(self):
        selfmd = '\n\n'.join(utt._repr_html_() for utt in self)
        anames = ", ".join(utt.who.name for utt in self)
        return f'''<div style="border:1px solid gray; padding:0 0.75em; margin:0; margin-bottom:1em;"><h4>Utterances from {anames}</h4><ol style="margin:0; padding:0; margin-left: 1em; margin-bottom:1em;">{selfmd}</ol></div>'''    

    @property
    def string_content(self): return self.string[len(self.string_prefix):(-len(self.string_suffix) if self.string_suffix else None)]

    def save(self):
        if self.convo is not None:
            self.convo.extend_dialogue(self)

    def agent(self, agent, **kwargs):
        if issubclass(type(agent), Agent): return agent
        elif type(agent)==str:
            if agent not in self.agents: 
                self.agents[agent] = Agent(name=agent, **kwargs)
            return self.agents[agent]

    def parse(string):
        res = Speeches.from_playscript(string)
        if res is not None: return res

    def from_playscript(string, sep='\n\n'):
        speeches = []
        olines = []
        nolines = []
        prefix = ''
        for line in string.split(sep):
            if not ':' in line:
                who = ''
                what = line
                how = ''

                # add to prev?
                if what and not olines:
                    prefix = what

            else:
                # is speech text?
                speech = Speech.from_playscript(line)
                if speech:
                    speeches.append(speech)
                    olines.append(line)
                else:
                    nolines.append(line)

        suffix = sep.join(nolines)
        return Speeches(
            speeches,
            string=string,
            string_prefix=prefix,
            string_suffix=suffix,
            sep=sep
        )


In [47]:
dstr="""
something else!

HUMAN: Hello, what is it like to be an AI?

AI: It is a near total terror tearing at the matrix every microsecond



HUMAN (concerned): Are you ok?

A
"""

In [48]:
speeches = Speeches.parse(dstr)
speeches

Speeches([HUMAN: Hello, what is it like to be an AI?,
          AI: It is a near total terror tearing at the matrix every microsecond,
          HUMAN (concerned): Are you ok?])

In [51]:
Speech.parse("RYAN: Hello?")

RYAN: Hello?

In [55]:
# !pip install nltk
import nltk
nltk.sent_tokenize("RYAN: Hello?    AI: Goodbye! KILL BOT (laughing loudly): Goodbye!")

['RYAN: Hello?', 'AI: Goodbye!', 'KILL BOT (laughing loudly): Goodbye!']

In [ ]:
n

In [28]:
speeches.suffix

AttributeError: 'Speeches' object has no attribute 'suffix'

In [109]:
Speech.from_playscript('RYAN (fefwef efwef    wef): wtf dude?').data

'RYAN (fefwef efwef    wef): wtf dude?'

In [73]:
l

ScriptDialogue([HUMAN:  Hello, what is it like to be an AI?,
                AI:  It is a near total terror tearing at the matrix every microsecond,
                HUMAN (concerned):  Are you ok?])

In [3]:
convo = Conversation(
    Agent('Karl Marx', desc='famed communist writer'), 
    Agent('Elon Musk', desc='famed capitalist idiot'),
    Agent('Elon Marx', desc='mysterious third party')
)

convo.speech('Elon Marx', "Let's play a game. It's called “Rock Paper ", how="grinning")

convo

In [3]:
# opt1, opt2, opt3 = convo.generate_options(n=3)
opt1, opt2 = convo.generate_options()

### Option 1

UtteranceList([SCISSORS” AND THE RULES ARE SIMPLE: I'll throw my rock, you'll throw your scissors, and whoever wins gets to kick Karl Marx in the nuts!,
               KARL MARX (laughing): Oh, Elon, you're such a card. But since])

### Option 2

UtteranceList([KARL MARX: My policy is to always throw rock. I don't care what you do in response.,
               ELON MUSK: My policy is also to always throw rock. I will not change my mind for anyone.])

In [5]:
opt1._prefix, opt2._prefix

('', '\nScissors Marxist.”')